In [11]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json
import requests
import numpy as np
import os
from dotenv import load_dotenv
from scipy.optimize import minimize, differential_evolution

load_dotenv()
px.set_mapbox_access_token(os.getenv("MAPBOX_TOKEN"))

In [24]:
# census tract svi ata
svi_df = pd.read_csv("https://svi.cdc.gov/Documents/Data/2022/csv/states/SVI_2022_US.csv", dtype = {"FIPS" :str})
#filter 
svi_df = svi_df[svi_df['STATE']=='Maryland'].reset_index(drop=True)
svi_df = svi_df[svi_df['COUNTY']=='Baltimore City'].reset_index(drop=True)

grocery_stores = pd.read_csv("https://opendata.arcgis.com/api/v3/datasets/85924b7086ef4506b4f2240d282a54c0_0/downloads/data?format=csv&spatialRefId=2248&where=1%3D1")
# lat lon
grocery_coords = list(zip(grocery_stores['latitude'], grocery_stores['longitude']))


In [25]:
# maryland boundary geojson
response = requests.get('https://opendata.arcgis.com/api/v3/datasets/2cb6e7dfc87047ccaa38f59d955d907b_2/downloads/data?format=geojson&spatialRefId=4326&where=1%3D1')
tracts = json.loads(response.text)

filtered_features = [feature for feature in tracts["features"] if feature["properties"]["COUNTYFP20"] == '510']
tracts["features"] = filtered_features

In [ ]:
(svi_df["E_TOTPOP"]/svi_df["E_HH"]).sort_values()

In [ ]:
fig = px.choropleth_mapbox(
    svi_df,
    geojson=tracts,
    locations="FIPS",
    featureidkey="properties.GEOID20",
    color="RPL_THEMES",  # EP_Noveh: veh percent,  RPL_THEMES: svi percentile
    color_continuous_scale="matter",
    range_color=(0, 1),
    mapbox_style="light",
    zoom=10.5,
    center={"lat": 39.29, "lon": -76.62},
    opacity=0.85,
    hover_data=["COUNTY", "RPL_THEMES", "E_TOTPOP", "E_HH", "E_NOVEH"],
    labels={
        "RPL_THEMES": "SVI National Percentile",
        "TOTPOP": "Total population",
        "EP_NOVEH": "Household without vehichles",
        "E_HH": "Household"
    },
    width=1000,
    height=640,
)
# fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0}, title = "Baltimore City")
# fig = fig.update_traces(
#     marker_line_width=0.1
# )

groceryTrace = px.scatter_mapbox(
    data_frame=grocery_stores,
    opacity=1,
    hover_name="storename",
    lat="latitude",
    lon="longitude",
    text="address",
)
groceryTrace.update_traces(mode="markers", marker=dict(color="#34eb64", size=10))

fig.add_trace(groceryTrace.data[0])

# fig.show()
fig.show()

In [36]:
import math
#thanks chatgpt
def haversine_distance(lat1, lon1, lat2, lon2):
    # distance between latitudes
    # and longitudes
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0

    # convert to radians
    lat1 = (lat1) * math.pi / 180.0
    lat2 = (lat2) * math.pi / 180.0

    # apply formulae
    a = (pow(math.sin(dLat / 2), 2) +
         pow(math.sin(dLon / 2), 2) *
             math.cos(lat1) * math.cos(lat2));
    rad = 3958.8
    c = 2 * math.asin(math.sqrt(a))
    return rad * c

In [42]:
tracts_ID = [feature["properties"]["GEOID20"] for feature in tracts["features"]]
tracts_lat = pd.to_numeric(
    [feature["properties"]["INTPTLAT20"] for feature in tracts["features"]]
)
tracts_lon = pd.to_numeric(
    [feature["properties"]["INTPTLON20"] for feature in tracts["features"]]
)

# has lat, lon, fips, popplation, household, SVI percentile, %no veh
tracts_locations = pd.DataFrame(
    {"FIPS": tracts_ID, "lat": tracts_lat, "lon": tracts_lon}
)
tracts_locations = tracts_locations.merge(
    svi_df.loc[:, ["FIPS", "E_TOTPOP", "E_HH", "RPL_THEMES", "EP_NOVEH", "E_NOVEH"]],
    on="FIPS",
)
# clean broken record
tracts_locations = tracts_locations[tracts_locations["E_HH"] > 1]

In [70]:
def flattened_to_pair(flattened_list):
    return [
        (flattened_list[i], flattened_list[i + 1])
        for i in range(0, len(flattened_list), 2)
    ]


def dist_to_closest_grocery(lat, lon, grocery_coords):
    return min(
        [
            haversine_distance(lat, lon, groc_lat, groc_lon)
            for groc_lat, groc_lon in grocery_coords
        ]
    )


def avg_dist_to_grocery(new_locations):
    # create tuples from list
    new_locations = flattened_to_pair(new_locations)
    new_grocery_coords = grocery_coords + new_locations
    scoring_metric = lambda x: dist_to_closest_grocery(
        x["lat"], x["lon"], new_grocery_coords
    )
    new_dists = tracts_locations.apply(scoring_metric, axis=1)
    return np.mean(new_dists)


def avg_dist_to_grocery_household(new_locations):
    # create tuples from list
    new_locations = flattened_to_pair(new_locations)
    new_grocery_coords = grocery_coords + new_locations
    scoring_metric = (
        lambda x: dist_to_closest_grocery(x["lat"], x["lon"], new_grocery_coords)
        * x["E_HH"]
    )
    new_dists = tracts_locations.apply(scoring_metric, axis=1)
    # weighted average by household
    return np.sum(new_dists)/np.sum(tracts_locations["E_HH"])


def avg_dist_to_grocery_noveh(new_locations):
    new_locations = flattened_to_pair(new_locations)
    new_grocery_coords = grocery_coords + new_locations
    scoring_metric = (
        lambda x: dist_to_closest_grocery(x["lat"], x["lon"], new_grocery_coords)
        * x["E_NOVEH"]
    )
    new_dists = tracts_locations.apply(scoring_metric, axis=1)
    #weighted average by household no veh
    return np.sum(new_dists) / np.sum(tracts_locations["E_NOVEH"])

In [ ]:
np.sum(tracts_locations["E_HH"])

In [ ]:
print(avg_dist_to_grocery([]))
print(avg_dist_to_grocery_household([]))
print(avg_dist_to_grocery_noveh([]))

In [ ]:
# initial = (40, -76)
num_stores = 3
bounds = [(39.1964, 39.374), (-76.712, -76.5295)] * num_stores

# Optimization using the minimize function from scipy
# options = {'maxiter': 1000000}
# result = (avg_dist_to_grocery, initial, method='dogleg', options=options, tol=1e-16)

result_HH = differential_evolution(avg_dist_to_grocery_household,  bounds, disp=True, tol=1e-12, workers=-1)
result_HH = flattened_to_pair(result_HH.x)
result_noveh = differential_evolution(avg_dist_to_grocery_noveh,  bounds, disp=True, tol=1e-12, workers=-1)
result_noveh = flattened_to_pair(result_noveh.x)
result_county = differential_evolution(avg_dist_to_grocery,  bounds, disp=True, tol=1e-12, workers=-1)
result_county = flattened_to_pair(result_county.x)


print(result_HH)
print(result_noveh)
print(result_county)


In [ ]:

result_df = pd.DataFrame(flattened_to_pair(result.x), columns=['lat', 'lon'])
result_df

In [ ]:
result_trace = px.scatter_mapbox(
    data_frame= result_df,
    opacity=1,
    lat='lat',
    lon='lon',
)
result_trace.update_traces(mode = "markers", marker = dict(color='red', size=12))

fig.add_trace(result_trace.data[0])

In [ ]:
print(avg_dist_to_grocery_household(result.x))

In [107]:
fig.write_html("./best_3_by_tracts.html")

In [111]:
result_HH = flattened_to_pair(result.x)